In [18]:
#Chainer V2 code! Check carefully!
import math
import numpy as np
import chainer
import chainer.links as L
import chainer.functions as F
from chainer import cuda
from chainer import optimizers
from chainer import training
from chainer.training import extensions
from chainer.cuda import cupy as xp
from chainer.dataset import convert
import six
from six.moves import queue
from chainer import serializers
#
from chainer.datasets import tuple_dataset
from PIL import Image
import numpy as np
import glob,os

In [2]:
#Define small blocks in residual net      
class BottleNeckA(chainer.Chain):
    def __init__(self, in_size, ch, out_size, stride=2):
        w = math.sqrt(2)
        super(BottleNeckA, self).__init__(
            conv1=L.Convolution2D(in_size, ch, 1, stride, 0, w, initial_bias=False),
            bn1=L.BatchNormalization(ch),
            conv2=L.Convolution2D(ch, ch, 3, 1, 1, w, initial_bias=False),
            bn2=L.BatchNormalization(ch),
            conv3=L.Convolution2D(ch, out_size, 1, 1, 0, w, initial_bias=False),
            bn3=L.BatchNormalization(out_size),

            conv4=L.Convolution2D(in_size, out_size, 1, stride, 0, w, initial_bias=False),
            bn4=L.BatchNormalization(out_size),
        )

    def __call__(self, x, train):
        h1 = F.relu(self.bn1(self.conv1(x)))
        h1 = F.relu(self.bn2(self.conv2(h1)))
        h1 = self.bn3(self.conv3(h1))
        h2 = self.bn4(self.conv4(x))

        return F.relu(h1 + h2)

        
class BottleNeckB(chainer.Chain):
    def __init__(self, in_size, ch):
        w = math.sqrt(2)
        super(BottleNeckB, self).__init__(
            conv1=L.Convolution2D(in_size, ch, 1, 1, 0, w, initial_bias=False),
            bn1=L.BatchNormalization(ch),
            conv2=L.Convolution2D(ch, ch, 3, 1, 1, w, initial_bias=False),
            bn2=L.BatchNormalization(ch),
            conv3=L.Convolution2D(ch, in_size, 1, 1, 0, w, initial_bias=False),
            bn3=L.BatchNormalization(in_size),
        )

    def __call__(self, x, train):
        h = F.relu(self.bn1(self.conv1(x)))
        h = F.relu(self.bn2(self.conv2(h)))
        h = self.bn3(self.conv3(h))

        return F.relu(h + x)

    
class Block(chainer.Chain):
    def __init__(self, layer, in_size, ch, out_size, stride=2):
        super(Block, self).__init__()
        links = [('a', BottleNeckA(in_size, ch, out_size, stride))]
        for i in range(layer-1):
            links += [('b{}'.format(i+1), BottleNeckB(out_size, ch))]

        for l in links:
            self.add_link(*l)
        self.forward = links

    def __call__(self, x, train):
        for name, _ in self.forward:
            f = getattr(self, name)
            x = f(x, train)

        return x

In [3]:
class ResNet(chainer.Chain):

    insize = 224

    def __init__(self):
        w = math.sqrt(2)
        super(ResNet, self).__init__(
            conv1=L.Convolution2D(3, 64, 7, 2, 3, w, initial_bias=False),
            bn1=L.BatchNormalization(64),
            res2=Block(layer=3, in_size=64, ch=64, out_size=256, stride=1),
            res3=Block(4, 256, 128, 512, 2),
            res4=Block(23, 512, 256, 1024, 2),
            res5=Block(3, 1024, 512, 2048, 2),
            fc=L.Linear(2048, 1000)
        )
        self.train = True

    def clear(self):
        self.loss = None
        self.accuracy = None

    def __call__(self, x, t):
        self.clear()
        h = self.bn1(self.conv1(x))
        h = F.max_pooling_2d(F.relu(h), 3, stride=2)
        h = self.res2(h, self.train)
        h = self.res3(h, self.train)
        h = self.res4(h, self.train)
        h = self.res5(h, self.train)
        h = F.average_pooling_2d(h, 7, stride=1)
        h = self.fc(h)

        if self.train:
            self.loss = F.softmax_cross_entropy(h, t)
            self.accuracy = F.accuracy(h, t)
            return self.loss
        else:
            return h

In [4]:
def data_manage_animefacedata_gpu(data_path="/home/koma/dataset/animeface-character-dataset/thumb/"):
    #Data path setup
   
    #data_path = "/home/koma/dataset/animeface-character-dataset/thumb/"
    #test_data_path = "/data/test/"
    
    #folders = glob.glob(data_path+"*")
    folders = sorted(os.listdir(data_path))
    cats = []#Categorys list
    all_data = []
    for folder in folders:
        if os.path.isfile(data_path + folder + "/" + "ignore"):
            print("Folder "+ folder + "is ignored!")
            continue
        else:
            cats.append(folder)
            label = folder
            img_filelist = glob.glob(data_path + folder + "/"+"*.png")
            for imgfile in img_filelist:
                all_data.append([imgfile,label])
    print("labels="+str(len(cats)))
    
    all_data = np.random.permutation(all_data)
            
    imageData = []
    labelData = []
    for PathAndLabel in all_data:
        img = Image.open(PathAndLabel[0])
        img = img.resize((160,160))
        label_id = cats.index(PathAndLabel[1])
        #print PathAndLabel[1]
        img = np.asarray(np.float32(img))
        if img.shape[2] != 3:
            continue
        img = np.reshape((img),(3,160,160))
        
        imageData.append(img)
        labelData.append(np.int32(label_id))
    
    threshold = np.int32(len(imageData)/8*7)
    train = tuple_dataset.TupleDataset(xp.asarry(imageData[0:threshold]), xp.asarry(labelData[0:threshold]))
    test  = tuple_dataset.TupleDataset(xp.asarry(imageData[threshold:]),  xp.asarry(labelData[threshold:]))
    return train,test
    
    
    #train_batch = np.ndarray((50,3,160,160),dtype=np.float32)
    #test_batch = np.ndarray((50,3,160,160),dtype=np.float32)
    

In [5]:
def data_manage_animefacedata(data_path="/home/koma/dataset/animeface-character-dataset/thumb/",in_size=224):
    #Data path setup
   
    #data_path = "/home/koma/dataset/animeface-character-dataset/thumb/"
    #test_data_path = "/data/test/"
    
    #folders = glob.glob(data_path+"*")
    folders = sorted(os.listdir(data_path))
    cats = []#Categorys list
    all_data = []
    for folder in folders:
        if os.path.isfile(data_path + folder + "/" + "ignore"):
            #print("Folder "+ folder + "is ignored!")
            continue
        else:
            cats.append(folder)
            label = folder
            img_filelist = glob.glob(data_path + folder + "/"+"*.png")
            for imgfile in img_filelist:
                all_data.append([imgfile,label])
    print("labels="+str(len(cats)))
    
    all_data = np.random.permutation(all_data) #Random the rank
            
    imageData = []
    labelData = []
    for PathAndLabel in all_data:
        img = Image.open(PathAndLabel[0])
        img = img.resize((in_size,in_size))
        label_id = cats.index(PathAndLabel[1])
        #print PathAndLabel[1]
        img = np.asarray(np.float32(img))
        if img.shape[2] != 3:
            continue
        img = np.reshape(img,(3,in_size,in_size))
        imageData.append(img)
        labelData.append(np.int32(label_id))
    
    threshold = np.int32(len(imageData)/8*7)
    train = tuple_dataset.TupleDataset(imageData[0:threshold], labelData[0:threshold])
    test  = tuple_dataset.TupleDataset(imageData[threshold:],  labelData[threshold:])
    return train,test
    
    
    #train_batch = np.ndarray((50,3,160,160),dtype=np.float32)
    #test_batch = np.ndarray((50,3,160,160),dtype=np.float32)
    

In [6]:
if __name__ == "__main__":
    
    #Check cuda environment and setup models
    cuda.check_cuda_available()
    from chainer.cuda import cupy as xp
    
    model = ResNet()
    cuda.get_device(0).use()
    model.to_gpu(0)
    print("Model Ready with GPU")

Model Ready with GPU


In [7]:
    #Setup optimizer
    optimizer = optimizers.Adam(alpha=0.0002, beta1=0.5)
    optimizer.setup(model)
    optimizer.add_hook(chainer.optimizer.WeightDecay(0.00001))
    print("Optimizer Sets!")

Optimizer Sets!


In [ ]:
    #Setup train data and test data
    train, test = data_manage_animefacedata()
    train_iter = chainer.iterators.SerialIterator(train,20)
    test_iter = chainer.iterators.SerialIterator(test,10,repeat=False,shuffle=False)
    print("Data Loaded!")

labels=149
Data Loaded!


In [ ]:
    sum_accuracy = 0
    sum_loss = 0
    epochnum = 10
    
    train_count = len(train)
    test_count = len(test)
    #while(train_iter.epoch < args.epoch):
    while(train_iter.epoch < epochnum):
        batch = train_iter.next()
        #print len(batch)
        print('train mean loss: {}, accuracy: {}'.format(
                sum_loss / train_count, sum_accuracy / train_count))
        # Reduce learning rate by 0.5 every 25 epochs.
        #if train_iter.epoch % 25 == 0 and train_iter.is_new_epoch:
        #   optimizer.lr *= 0.5
        #   print('Reducing learning rate to: ', optimizer.lr)

        #x_array, t_array = convert.concat_examples(batch, args.gpu)
        x_array, t_array = convert.concat_examples(batch, 0)
        x = chainer.Variable(x_array)
        t = chainer.Variable(t_array)
        optimizer.update(model, x, t)
        sum_loss += float(model.loss.data) * len(t.data)
        sum_accuracy += float(model.accuracy.data) * len(t.data)
        
        if train_iter.is_new_epoch:
            print('epoch: ', train_iter.epoch)
            print('train mean loss: {}, accuracy: {}'.format(
                sum_loss / train_count, sum_accuracy / train_count))
            # evaluation
            sum_accuracy = 0
            sum_loss = 0
            #model.predictor.train = False
            for batch in test_iter:
                x_array, t_array = convert.concat_examples(batch, 0)
                x = chainer.Variable(x_array)
                t = chainer.Variable(t_array)
                loss = model(x, t)
                sum_loss += float(loss.data) * len(t.data)
                sum_accuracy += float(model.accuracy.data) * len(t.data)

            test_iter.reset()
            #model.predictor.train = True
            print('test mean  loss: {}, accuracy: {}'.format(
                sum_loss / test_count, sum_accuracy / test_count))
            sum_accuracy = 0
            sum_loss = 0

            # Save the model and the optimizer
            print('save the model')
            serializers.save_npz('mlp.model', model)
            print('save the optimizer')
            serializers.save_npz('mlp.state', optimizer)

train mean loss: 0, accuracy: 0
train mean loss: 0.00770275593025, accuracy: 0.00018540836468
train mean loss: 0.0147904352471, accuracy: 0.00027811254702
train mean loss: 0.0227133770454, accuracy: 0.00027811254702
train mean loss: 0.0295316413787, accuracy: 0.000556225100947
train mean loss: 0.0375218054136, accuracy: 0.000741633465627
train mean loss: 0.0453690898157, accuracy: 0.000927041830307
train mean loss: 0.0525880600355, accuracy: 0.00101974601265
train mean loss: 0.061549985222, accuracy: 0.00120515437733
train mean loss: 0.0702678822406, accuracy: 0.00129785855967
train mean loss: 0.0783674036417, accuracy: 0.00157597111359
train mean loss: 0.0861707803018, accuracy: 0.00157597111359
train mean loss: 0.0937506743, accuracy: 0.00166867529593
train mean loss: 0.101254251366, accuracy: 0.00194678784986
train mean loss: 0.109708738181, accuracy: 0.00194678784986
train mean loss: 0.118082964996, accuracy: 0.0020394920322
train mean loss: 0.126224240481, accuracy: 0.002317604586

train mean loss: 1.09200262588, accuracy: 0.0201168077128
train mean loss: 1.09892995848, accuracy: 0.0204876244422
train mean loss: 1.10762090383, accuracy: 0.0205803286245
train mean loss: 1.11506465855, accuracy: 0.0206730328069
train mean loss: 1.12433974202, accuracy: 0.0206730328069
train mean loss: 1.13141997946, accuracy: 0.0207657369892
train mean loss: 1.13864117141, accuracy: 0.0209511453539
train mean loss: 1.14633401839, accuracy: 0.0210438495362
train mean loss: 1.15456403412, accuracy: 0.0210438495362
train mean loss: 1.16246761334, accuracy: 0.0210438495362
train mean loss: 1.17086626596, accuracy: 0.0210438495362
train mean loss: 1.17882456629, accuracy: 0.0210438495362
train mean loss: 1.18671222647, accuracy: 0.0211365537186
train mean loss: 1.19529220363, accuracy: 0.0212292579009
train mean loss: 1.20321146317, accuracy: 0.0213219620832
train mean loss: 1.21108510601, accuracy: 0.0215073704479
train mean loss: 1.21862392845, accuracy: 0.0216927788126
train mean los

train mean loss: 2.18302191072, accuracy: 0.0424585157811
train mean loss: 2.1906189774, accuracy: 0.0424585157811
train mean loss: 2.19815422101, accuracy: 0.0424585157811
train mean loss: 2.20557190727, accuracy: 0.0425512199634
train mean loss: 2.21386928847, accuracy: 0.0426439241458
train mean loss: 2.22109917373, accuracy: 0.0429220366997
train mean loss: 2.22844679716, accuracy: 0.0431074450644
train mean loss: 2.23693215483, accuracy: 0.0431074450644
train mean loss: 2.24458789808, accuracy: 0.0431074450644
train mean loss: 2.25285125837, accuracy: 0.0431074450644
train mean loss: 2.26136107368, accuracy: 0.0432001492467
train mean loss: 2.26883518737, accuracy: 0.0433855576114
train mean loss: 2.27600396658, accuracy: 0.0436636701653
train mean loss: 2.28359248758, accuracy: 0.0437563743476
train mean loss: 2.29148541786, accuracy: 0.04384907853
train mean loss: 2.29934789191, accuracy: 0.0439417827123
train mean loss: 2.30707791071, accuracy: 0.0440344868947
train mean loss: 

train mean loss: 3.25708354758, accuracy: 0.065171040627
train mean loss: 3.2637322096, accuracy: 0.0652637448094
train mean loss: 3.27131758941, accuracy: 0.0654491531741
train mean loss: 3.27840230612, accuracy: 0.0655418573564
train mean loss: 3.28605161668, accuracy: 0.0658199699103
train mean loss: 3.29538637575, accuracy: 0.0659126740927
train mean loss: 3.30355629064, accuracy: 0.066005378275
train mean loss: 3.31125189954, accuracy: 0.0662834908289
train mean loss: 3.31809023462, accuracy: 0.0664688991936
train mean loss: 3.32594086709, accuracy: 0.0666543075583
train mean loss: 3.33429212863, accuracy: 0.066839715923
train mean loss: 3.34182321876, accuracy: 0.0669324201053
train mean loss: 3.34923223867, accuracy: 0.0672105326592
train mean loss: 3.3566817126, accuracy: 0.0674886452132
train mean loss: 3.36403542346, accuracy: 0.0675813493955
train mean loss: 3.37109074266, accuracy: 0.0677667577602
train mean loss: 3.37829899056, accuracy: 0.068230278665
train mean loss: 3.3

train mean loss: 0.159403582239, accuracy: 0.00519143428011
train mean loss: 0.165772801413, accuracy: 0.00556225100947
train mean loss: 0.172111563038, accuracy: 0.00584036356339
train mean loss: 0.178918880053, accuracy: 0.00602577192807
train mean loss: 0.185751714288, accuracy: 0.00611847611041
train mean loss: 0.192789647293, accuracy: 0.00630388447509
train mean loss: 0.19972855713, accuracy: 0.00648929283977
train mean loss: 0.20645799369, accuracy: 0.00686010956913
train mean loss: 0.21364028763, accuracy: 0.00704551793381
train mean loss: 0.220691424167, accuracy: 0.00732363048774
train mean loss: 0.227273716214, accuracy: 0.00741633467008
train mean loss: 0.233897187684, accuracy: 0.00760174303476
train mean loss: 0.242348769359, accuracy: 0.00778715139944
train mean loss: 0.251112215886, accuracy: 0.00787985558178
train mean loss: 0.258124079996, accuracy: 0.00825067231114
train mean loss: 0.264273371457, accuracy: 0.00843608067582
train mean loss: 0.270678621962, accuracy: 

...

train mean loss: 2.05066512797, accuracy: 0.377584130043
train mean loss: 2.05477734563, accuracy: 0.378325763502
train mean loss: 2.05816358458, accuracy: 0.37906739696
train mean loss: 2.06326524045, accuracy: 0.379530917865
train mean loss: 2.06819701551, accuracy: 0.380087142973
train mean loss: 2.07125583614, accuracy: 0.380828776432
train mean loss: 2.07504904369, accuracy: 0.381663114038
train mean loss: 2.07889236223, accuracy: 0.382590155848
train mean loss: 2.08282562693, accuracy: 0.383424493454
train mean loss: 2.08689087699, accuracy: 0.384258831061
train mean loss: 2.09098854818, accuracy: 0.384907760317
train mean loss: 2.09427660716, accuracy: 0.385649393775
train mean loss: 2.09769711477, accuracy: 0.386669139788
train mean loss: 2.10126608994, accuracy: 0.387596181598
train mean loss: 2.10567268813, accuracy: 0.388337815056
train mean loss: 2.10819082728, accuracy: 0.389542969365
train mean loss: 2.11373704506, accuracy: 0.389913786094
train mean loss: 2.11818680879, 

train mean loss: 0.337702255933, accuracy: 0.0834337631114
train mean loss: 0.342050103292, accuracy: 0.0839899882192
train mean loss: 0.346354251555, accuracy: 0.0846389174749
train mean loss: 0.350210964155, accuracy: 0.0851951425827
train mean loss: 0.353256232703, accuracy: 0.0858440718384
train mean loss: 0.357526085115, accuracy: 0.0865857052971
train mean loss: 0.360210559184, accuracy: 0.0878835638084
train mean loss: 0.364182402297, accuracy: 0.0885324930641
train mean loss: 0.367939436614, accuracy: 0.0891814223197
train mean loss: 0.371492874018, accuracy: 0.0899230557785
train mean loss: 0.374798639674, accuracy: 0.0906646892372
train mean loss: 0.378338092448, accuracy: 0.0916844352498
train mean loss: 0.382369925892, accuracy: 0.0927968854655
train mean loss: 0.38671225605, accuracy: 0.0935385189242
train mean loss: 0.389556216168, accuracy: 0.0947436732325
train mean loss: 0.392932534892, accuracy: 0.0960415317438
train mean loss: 0.396342691726, accuracy: 0.097061277756

train mean loss: 0.857450073108, accuracy: 0.205988690514
train mean loss: 0.861724757105, accuracy: 0.206452211419
train mean loss: 0.86520813344, accuracy: 0.207471957431
train mean loss: 0.869852725399, accuracy: 0.208306295038
train mean loss: 0.872734191078, accuracy: 0.209511449346
train mean loss: 0.876053519429, accuracy: 0.210531195359
train mean loss: 0.879585940546, accuracy: 0.211458237168
train mean loss: 0.884123927329, accuracy: 0.212014462276
train mean loss: 0.887970348168, accuracy: 0.212756095735
train mean loss: 0.893306317758, accuracy: 0.213497729194
train mean loss: 0.897219662009, accuracy: 0.2143320668
train mean loss: 0.901953150259, accuracy: 0.21525910861
train mean loss: 0.904666179964, accuracy: 0.216464262918
train mean loss: 0.907941624664, accuracy: 0.217484008931
train mean loss: 0.911688689915, accuracy: 0.218318346537
train mean loss: 0.916019462728, accuracy: 0.218967275793
train mean loss: 0.920024394901, accuracy: 0.219987021806
train mean loss: 0

train mean loss: 1.42852242048, accuracy: 0.321127283295
train mean loss: 1.43201998766, accuracy: 0.321868916754
train mean loss: 1.43651433547, accuracy: 0.322610550212
train mean loss: 1.44145591206, accuracy: 0.323352183671
train mean loss: 1.4441231898, accuracy: 0.324279225481
train mean loss: 1.44736244779, accuracy: 0.325298971493
train mean loss: 1.45010983766, accuracy: 0.326411421709
train mean loss: 1.45509672726, accuracy: 0.327060350965
train mean loss: 1.45965973927, accuracy: 0.327523871869
train mean loss: 1.46282398105, accuracy: 0.328450913679
train mean loss: 1.46696094323, accuracy: 0.329099842935
train mean loss: 1.47028828967, accuracy: 0.330026884744
train mean loss: 1.47332586459, accuracy: 0.33113933496
train mean loss: 1.47575032317, accuracy: 0.332437193471
train mean loss: 1.47983238299, accuracy: 0.332993418579
train mean loss: 1.48405311237, accuracy: 0.333642347835
train mean loss: 1.48823923029, accuracy: 0.334198572943
train mean loss: 1.49220997138, a

train mean loss: 1.98915840621, accuracy: 0.440159451734
train mean loss: 1.99348633135, accuracy: 0.44080838099
train mean loss: 1.99605649203, accuracy: 0.441828127002
train mean loss: 1.99980193496, accuracy: 0.442847873015
train mean loss: 2.00275129214, accuracy: 0.443960323231
train mean loss: 2.00492198821, accuracy: 0.445072773447
train mean loss: 2.00867809069, accuracy: 0.446185223662
train mean loss: 2.01275687051, accuracy: 0.44674144877
train mean loss: 2.01550786131, accuracy: 0.447761194783
train mean loss: 2.01841253981, accuracy: 0.448780940795
train mean loss: 2.02355674633, accuracy: 0.449429870051
train mean loss: 2.02568236522, accuracy: 0.450635024359
train mean loss: 2.028790041, accuracy: 0.451562066169
('epoch: ', 9)
train mean loss: 2.03139508346, accuracy: 0.45285992468
test mean  loss: 3.01780896511, accuracy: 0.27378641421
save the model
save the optimizer
train mean loss: 0, accuracy: 0
train mean loss: 0.00220638967217, accuracy: 0.00129785851132
train me